In [1]:
# Import qiskit and modules
import matplotlib.pyplot as plt
import numpy as np
import math
import random
import time

from math import pi

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile, assemble
from qiskit import Aer
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer
from qiskit.circuit.library import TwoLocal

from scipy.optimize import minimize

In [7]:
from AtomLoader import *

In [8]:
################ Tutorial Block ################
# Initializing a three-qubit quantum state
desired_vector = [
    math.cos(np.pi/4),
    0,
    math.cos(np.pi/4),
    0,
    0,
    0,
    0,
    0]


q = QuantumRegister(3)

qc = QuantumCircuit(q)

qc.initialize(desired_vector, [q[0],q[1],q[2]])
qc.draw()

backend = BasicAer.get_backend('statevector_simulator')
job = backend.run(transpile(qc, backend))
qc_state = job.result().get_statevector(qc)
qc_state

state_fidelity(desired_vector,qc_state)

1.0

In [9]:
def training_circuit(parameters, atomic_descriptors):
    
    qc = QuantumCircuit(M)
    qc_descriptor = QuantumCircuit(M)
    
    list_eta = parameters[0:M]
    list_ksi = parameters[M:M + (depth+1)*M*2] ## twolocal have (depth+1)*M*2 parameters
    
    # for i in range(M):
    for i, descriptor in enumerate(atomic_descriptors):
        qc_descriptor.u(
            descriptor[0],
            descriptor[1],
            0,
            i
            ) #should be replaced to theta(list_eta[i]), phi(list_ksi[i])
        #Young Oh should read this!

    twolocal = TwoLocal(num_qubits=M, reps=depth, rotation_blocks=['ry','rz'], 
                   entanglement_blocks='cx', entanglement='circular', parameter_prefix='ξ', insert_barriers=True)
    twolocal = twolocal.bind_parameters(list_ksi)
    
    qc += qc_descriptor
    qc.barrier()

    qc += twolocal
    qc.barrier()

    #Observable
    qc.z(0)

    qc.barrier()

    qc += twolocal.inverse()
    qc.barrier()

    qc += qc_descriptor.inverse()
    qc.barrier()
    
    return qc

In [57]:
def calculate_loss_function(parameters, molecule_index):

    backend = Aer.get_backend('aer_simulator')

    energy = 0
    
    list_eta = parameters[0:M]
    atom_data = AtomLoader(list_eta, idx=[molecule_index])
    ground_energy_label = atom_data[molecule_index]['ground_energy']
    descriptors = atom_data[molecule_index]['descriptor']
    n_atoms = len(atom_data[molecule_index]['descriptor'])

    for i, atomic_descriptors in enumerate(descriptors):

        qctl = QuantumRegister(M)
        qc = ClassicalRegister(M)
        circ = QuantumCircuit(qctl, qc)

        circ += training_circuit(parameters, atomic_descriptors)

        circ.save_statevector()
        t_circ = transpile(circ, backend)
        qobj = assemble(t_circ)
        job = backend.run(qobj)

        result = job.result()
        outputstate = result.get_statevector(circ, decimals=100)
        o = outputstate

        energy += np.real(o[0]) ## <0|GdWdOWG|0> is picking first component of GdWdOWG|0>. I don't think square is necessary

    loss = ((energy - ground_energy_label)/n_atoms)**2

    return loss

In [62]:
M = 2 #Number of descriptors (features) = Number of qubits
N = 3 #Number of atoms = Number of circuits
depth = 1 #Number of repetitions of layer
len_param = M*(2*depth+3) #Number of total parameters (eta + ksi)

print("Number of total parameters:", len_param)
print()

random_indexes = [str(x) for x in (np.random.choice(len(qm9), 10) + 1)]

x0=[float(random.randint(0,3000))/1000 for i in range(0, len_param)]

start = time.time()

for i,molecule_index in enumerate(random_indexes):
    print("Current index of molecule:", i+1)
    
    print(x0)
    out = minimize(calculate_loss_function,
    x0,
    method="COBYLA",
    options={'maxiter':15},
    args=(molecule_index)
    )
    
    x0 = out.x
    print(x0)
    
    print("Current Loss Function:",out['fun'])
    print()

end = time.time()

print(f"Calculation time: {end-start}s\n")

print(out)

print(out['x'])


Number of total parameters: 10

Current index of molecule: 1
[0.966, 2.663, 2.126, 0.76, 2.985, 1.262, 1.079, 2.176, 2.34, 0.234]
[2.10459907 3.9628171  4.39148447 0.3949704  2.233743   0.59805697
 0.06391321 2.176      2.34       0.234     ]
Current Loss Function: 304808.19104044617

Current index of molecule: 2
[2.10459907 3.9628171  4.39148447 0.3949704  2.233743   0.59805697
 0.06391321 2.176      2.34       0.234     ]
[1.61279045 4.9628171  4.48161923 1.3949704  3.233743   0.59805697
 1.06391321 2.176      2.34       0.234     ]
Current Loss Function: 447097.13292880956

Current index of molecule: 3
[1.61279045 4.9628171  4.48161923 1.3949704  3.233743   0.59805697
 1.06391321 2.176      2.34       0.234     ]
[1.60854998 4.96172501 4.93557372 1.31308601 3.11574169 0.44549509
 2.06519967 2.176      2.34       0.234     ]
Current Loss Function: 833997.4022436055

Current index of molecule: 4
[1.60854998 4.96172501 4.93557372 1.31308601 3.11574169 0.44549509
 2.06519967 2.176      